# CSCI E-25    
## Introduction to Machine Learning and Linear Models

### Steve Elston   

## Introduction to Linear Models 

The concept of the linear model is the basis of many statistical and machine learning models. Further, an understanding of linear models is a good basis for understand many other types of statistical and machine learning models.   

In this lesson we will focus on linear classification models, but the lessons drawn from this discussion can be applied to many other types of ML models. By developing an understanding of linear models, you are building a foundation to understand many other machine learning models. Nearly all machine learning methods suffer from the same problems, including over-fitting and mathematically unstable fitting methods. Understanding these problems in the linear regression context will help you work with other machine learning models.     

The method of regression is one of the oldest and most widely used analytics methods. The goal of regression is to produce a model that represents the **best fit** to some observed data. Typically the model is a function describing some type of curve (lines, parabolas, etc.) that is determined by a set of parameters (e.g., slope and intercept). *Best fit* means that there is an optimal set of parameters which minimize an error criteria we choose.     

Many machine learning models, including some of the latest deep learning methods, are a form of regression. **Linear regression** is the foundational form of regression. Linear regression minimizes squared error of the predictions of the dependent variable using the values of the independent variables. This approach is know as the **method of least squares**.   

Linear classifiers are an extension of linear regression. The linear output is transformed to a categorical probability distribution. The category with the highest probability is typically selected as the final result.  

## Load and Prepare MNIST image Data    

We will now work through an example of using a linear classifier for image classification. For this example we will use the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) image data set. MNIST is a commonly used benchmark standard data set used for image classification research. The dataset is comprised of $28 \times 28$ images of hand written digits in the set $[0-9$. There are 60,000 training images and labels and 10,000 test images and labels.            

To get started, execute the code in the cell below to import the packages you will require for this example.  

In [2]:
import numpy as np
import pandas as pd
import numpy.random as nr
import matplotlib.pyplot as plt
import keras
from keras.datasets import mnist
import keras.utils.np_utils as ku
import sklearn.linear_model as sklm
import sklearn.metrics as metrics
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
import skimage.feature as feature

/var/folders/tt/z2dnsthj301301nmslkrspm40000gp/T/ipykernel_41363/2627722997.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import sklearn
sklearn.__version__

The MNIST dataset is built into Keras with the training and test subsets of images and labels returned in lists. Execute the code in the cell below to load these subsets. 

> **Note:** The MNIST data contain simple images of hand written digits. These images are properly cropped and have nearly binary light (digit) and dark (background region) areas. No significant adjustment or transformation of these images is required. 

In [ ]:
(raw_train_images, train_labels), (raw_test_images, test_labels) = mnist.load_data()
print(raw_train_images.shape)

> **Exercise 4-1:** It is useful to get a feeling for what this image data really looks like. On a $5 \times 5$ grid display the first 25 gray-scale training images. Give each image display a title with the label for that image or case. 

In [ ]:
fig, ax = plt.subplots(5, 5, figsize=(15,15))
ax = ax.flatten()

## Your code goes below




> Examine the images and the labels. What problems can you foresee when a machine learning algorithm attempts to learn to classify the digits shown in these images? Your answer need only be 1 to 3 well chosen sentences.             
> **End of exercise.**

> **Answer:**     

In order to perform machine learning with image data you must transform the data to a structure with all the **features** for each image in one row of a **model matrix**. The linear model can then be written:      

$$X b = y$$

where:    
$X$ is the model matrix with the features values for each image in the rows.    
$b$ is the **coefficient vector**, with one coefficient per feature.   
$y$ is the vector of the **labels** which encode the categories of the objects in the images.

The goal is to estimate the vector of **coefficients**, **parameters** or **weights** to **minimize errors** in the the prediction of the label, $y$, given the model matrix, $X$. In machine learning terminology, we say that the model **learns the weights**, $b$, to minimize the errors.     

For this example, we will use the values of the pixels as our feature values. This requires the 2-dimensional images be **flattened** into feature vectors. This concept is illustrated in the figure below.   

<img src="img/FlatteningImge.JPG" alt="Drawing" style="width:600px; height:300px"/>
<center>Flattening an image to a feature vector</center>

> **Exercise 4-2:** You will now flatten the the $28 \times 28$ images to feature vectors. Do the following:    
> 1. Print the shape of the training image array, noticing that each image is a 2-dimensional sub-array.   
> 2. Flatten the images to an array of 60,000 rows using [numpy.reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html). The image arrays are 8 bit integers. Convert them to floating point in the range $[0.0 - 1.0]$, which will normalize the feature values.    
> 3. Print the shape of the flattened image array.   
> 4. Apply the same transformations to the test image array and print the shape of the resulting array.  

In [ ]:
## Your code goes below





> Examine the dimensions of the flattened arrays. How many features will your model have? Your answer should show a simple numeric calculation.    
> **End of exercise.**

> **Answer:**     

It can be instructive to look at the feature matrix. Execute the code in the cell below and examine the results.  

In [ ]:
def plot_feature_sample(img, nrows=512):
    fig, ax = plt.subplots(figsize=(15,9))
    p = ax.imshow(img[:nrows,:])
    plt.ylabel('Flattened image number')
    plt.xlabel('Features')
    plt.title('Flattened images')
    cb = plt.colorbar(p)
    _=cb.set_label('Intensity')

print('train_images.shape = ' + str(train_images.shape))
plot_feature_sample(train_images)    

> **Exercise 4-3:** Examine the feature columns in the array displayed above. Notice the differing ranges of values in the features. What does this tell you about the scale differences and the problems this might create in model training? Your answer need only be 1 to 3 well chosen sentences.           

> **Answer:**     

The significant difference in scale or value range of the features or pixel locations can lead to problems with model generalization. The coefficients computed will be affected by scale which will affect model performance and generalization. For example, consider that a bit of noise in some images presented to the classifier in production can lead to a high probability of erroneous classifications only as a result of scale differences. 

> **Note:** The behavior of low variance (small scale) features seen in this example may arise from the way the digit images are cropped. There are several approaches we could try here. 
> 1. Simply filter features with low variance (or small scale). The results may be a feature set with higher information content per feature.         
> 2. Transform the features so that they are all at the same scale.      
> 3. Once one of the above methods are applied to the features, dimensionality reduction might be useful. For linear models principle compomonent analysis (PCA) will typically be used.   
> **In other words, you must expolore and understand the nature of the data before applying any preprocessing step!**        

There are a number of methods we could use to deal with the scale problem. In this case, we will try the simplest possible approach, mean-variance scaling. Mean-variance scalling centers, or zero means, each feature. The centered feature values are then devided by the standard deviation, $\sigma$. For each feature $x$ we can express mean-variance scaling as follows: 

$$x_{scaled} = \frac{x - \bar{x}}{\sigma(x)}$$

Once the mean-varaince scaling is applied all features have zero mean and unit variance.   

The code in the cell below follows the usual process for mean-variance scaling:   
1. The [sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) is instantiated and fit to the traning data.    
2. The scaler is apply to the training data.     
3. The same scaler is apply to the test data. Note that it is important to use only the scalar learned from the training data to avoid bias.     
Execute the code in the cell below.  

In [ ]:
scalar = StandardScaler().fit(train_images)
train_images = scalar.transform(train_images)
test_images = scalar.transform(test_images)

## A Linear Model        

With the feature arrays prepared, it is time to construct the machine learning model. The code in the cell below defines the linear **logistic regression** model object and fits it to the training data. Here we use the Scikit-Learn [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) function. The model definition has several key aspects:   
- L2 regularization with a hyperparameter $C = 10.0$.  
- Since there are 10 categories of digits, the multinomial probability distribution is used.    
- An efficient solver for the system of linear equations is selected.    

The `fit` method with the arguments of the model (feature) matrix and the label vector.  

If you are just learning to use Scikit-Learn it is useful to know that the general approach is used for all machine learning models available in the package. A model object is defined including the values of hyperparameters. A fit method is used to compute model parameters or weights using the model matrix and labels as the arguments. The model matrix has the features in the columns, and the cases, in this case flattened images, in the rows. You can find a [Getting Started Guide](https://scikit-learn.org/stable/getting_started.html) in the Scikit-Learn documentation.   

The code in the row below constructs the model object and fits a model. You can see that some `l2` regularization is applied with an hyperparameter of 0.1. Since this is a mutli-class classification problem, the `multi_class='multinomial'` argument is used. Now, execute the code in the cell below to create a model object.  

> **Note:** Depending on your environment, you may need to change the solver to achieve convergence of the linear model. If you encounter this problem, uncomment the line of code below, and comment out the other line. 

In [ ]:
C=0.1
LinearClassifier = sklm.LogisticRegression(penalty='l2', C=C, multi_class='multinomial', solver ='newton-cg').fit(train_images, train_labels)

> **Exercise 4-4:** The logistic regression model computes probabilities each of the categories for each case. You will now investigate an example by the following steps:    
> 1. Apply the `predict_proba` method with the test images to the trained classifier method. 
> 2. Display the first 10 rows of the resulting array of probabilities.    
> 3. Sum the probabilities in the array across the classes.  

In [ ]:
## Your code goes below



> Answer the following questions in 1 to 3 well chosen sentences:  
> 1. Examine the probabilities of the categories for each of the 10 cases. Is there generally a category with the highest probability? Are there cases where another cases has a reasonably high probability?   
> 2. Given your answers to the foregoing question, do you expect this classifier to make errors in identifying the categories?   
> 3. Are these proper probability distributions, in the sense that they sum to 1.0 for each case?  
> **End of exercise.**

> **Answers:**
> 1.    
> 2.     
> 3.    

While it is useful to understand how the linear model algorithm computes probabilities for the categories of digits, for most applications we really only want to know the most probable category. The algorithm is quite simple; pick the category with the highest probability. Scikit-Learn provides the `predict` method that computes the probabilities are returns the category with the highest probability. Execute the code in the cell below to see an example.   

In [ ]:
class_predictions = LinearClassifier.predict(test_images)
class_predictions[:10]

## Evaluate the Model       

Now that you have a model and made predictions it is time to evaluate the model. The [Scikit-Learn metrics package](https://scikit-learn.org/stable/modules/model_evaluation.html) contains numerous functions for evaluating different types machine learning models. 

A widely used metric for evaluating classifiers; the number of correctly classified cases divided by all cases:   

$$Accuracy = \frac{TP}{TP + FP + TN + FN}$$    

Where, $TP$ are the true positives, $FP$ are the false positives, and $FN$ are the false negatives. In the **multi-class** case, we count all elements on the diagonal of the confusion matrix as true positive (TP). All cases on the diagonal are corrected classified in one category or another. Thus for computing average accuracy for the multi-class case there are only true positive cases in the numerator of the formula. You can find more details in [this review paper](https://arxiv.org/abs/2008.05756).    

In this case we will focus on evaluation of the multi-class classifier. The [sklearn.metrics.accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score) computes the accuracy given the actual labels and the predicted values. Execute the code in the cell below and examine the result.         

In [ ]:
print(metrics.accuracy_score(test_labels, class_predictions))

The overall accuracy of the model seems reasonably good. However, one must be extremely careful when evaluating any machine learning model. Any single metric can be quite misleading. It is good practice to look at several views of model performance.   

The **confusion matrix** can be a powerful tool for evaluating classifiers. The confusion matrix is a 2-dimensional array with the label values on vertical axis and the predicted values on the horizontal axis. The count of correctly classified cases for each category are along the diagonal. Counts or incorrectly classified cases are found off the diagonal. 

The confusion matrix can be computed and displayed numerically. For large numbers of categories a visualization of the confusion matrix can be useful. By studying the confusion matrix one can identify many problems which would not be apparent from one to two simple metrics. For example, accuracy of a classifier might seem quite high, but it could be misclassifying all members of some category, while doing well with other categories. Only by examination of the confusion matrix can such problems be discovered.     

The code in the cell does the following:  
1. Computes the multi-class confusion matrix using the [sklearn.metrics.multilabel_confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.multilabel_confusion_matrix.html?highlight=multilabel_confusion_matrix) function. 
2. displays the confusion matrix as a Pandas data frame to improve formatting.   
2. Displays the log values of the confusion matrix as an image or heat map. The logarithm is used in this case since the off-diagonal terms are quite small compared to the diagonal terms. A 1 is added to all terms to allow computation of the logarithm, adding a small but negligible bias.     

Execute the code and examine the results. 

In [ ]:
confusion_matrix = metrics.confusion_matrix(test_labels, class_predictions)   
print(pd.DataFrame(confusion_matrix))
p = plt.imshow(np.log(np.divide(confusion_matrix + 1.0, np.sum(confusion_matrix, axis=1))))
cb = plt.colorbar(p)
_=cb.set_label('Log count')

Some other commonly used [performance metrics for classifiers](https://en.wikipedia.org/wiki/Precision_and_recall) are **Precision** and **Recall**. For binary classifiers these metrics are defined:   

\begin{align}
Recall &= \frac{TP}{TP + FN}\\
Precision &=  \frac{TP}{TP + FP}
\end{align} 

You can think of recall as the fraction of positive cases correctly classified, also know as the **sensitivity** of the classifier. The precision or **positive predictive value** is the probability that a positive case can be correctly classified.  

The above formulas cannot be directly applied to the multi-class classification case one typically encounters in CV applications. For this situation we compute class-specific precision and recall from the multi-class confusion matrix using [sklearn.metrics.precision_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score) and [sklearn.metrics.recall_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score). 

The [class specific precision and recall are calculated from the confusion matrix](https://medium.com/data-science-in-your-pocket/calculating-precision-recall-for-multi-class-classification-9055931ee229#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6ImEyOWFiYzE5YmUyN2ZiNDE1MWFhNDMxZTk0ZmEzNjgwYWU0NThkYTUiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJuYmYiOjE2NzM0ODI2NzQsImF1ZCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMjAwNzU5NTcyMzgwNjA4Nzk4OSIsImVtYWlsIjoic3RlcGhlbi5lbHN0b25AZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImF6cCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsIm5hbWUiOiJTdGVwaGVuIEVsc3RvbiIsInBpY3R1cmUiOiJodHRwczovL2xoMy5nb29nbGV1c2VyY29udGVudC5jb20vYS9BRWRGVHA0YnJDbXJFVFpldW9LeGhqWS01WEFVR2hqU3gxRDdQWmpJVE9RaD1zOTYtYyIsImdpdmVuX25hbWUiOiJTdGVwaGVuIiwiZmFtaWx5X25hbWUiOiJFbHN0b24iLCJpYXQiOjE2NzM0ODI5NzQsImV4cCI6MTY3MzQ4NjU3NCwianRpIjoiMzc1ZjNiMTdmMDA4Mjg4NWQ4ZjgyODlhMzNlYmIyMTVkMjg0NmZjNCJ9.RpejRCenuDVzVPk5hAweRUqR4Pc-pJFRigIIW27mHf7Q0HqiO9CvkLJBu5vSvBoERBw2mdnaHpzpBNtJK3lRT8oPn-KgqX-Utol6lSrWLVGuuefsx4cHdz5CR0q6BOvy4CXk4dgeTpca6Psl51UNeuaV73hs-8bvROXzPoJACRxI0ykCNsf5nzcVysSiv5GOjckj0IfovXeh58N0nj5w4msaRvLUvThGDR7Id35M9JZmfKZimYwFSCC7dNn2Feq6KmkfdxU1U77jGRk9ZZfYSJk0kqIAWTOIh4O2flAwjzpX6ywwgLsBoIrIf1CxAG1d5F79zCapLfDCyOTB4JOVKQ), $C$. These metrics are weighted averages over the correctly and incorrectly classified cases for each label category.       
- The **class specific precision** for the $i$th class is the number of cases correctly classified divided by the sum of the $i$th row:         
$$precision_i = \frac{C_{i,i}}{\sum_j C_{i,j}} = \frac{TP_i}{\sum_j C_{i,j}}$$
- The **class specific recall** for the $j$th class is the number of cases correctly classified divided by the sum of the $j$th column:         
$$recall_j = \frac{C_{j,j}}{\sum_i C_{i,j}} = \frac{TP_j}{\sum_i C_{i,j}}$$

The code in the cell below computes and displays the class specific precision and recall for the classifier just constructed. Execute this code and examine the results.  

In [ ]:
unique_labels, label_counts = np.unique(test_labels, return_counts=True)
class_precision = metrics.precision_score(test_labels, class_predictions, labels=unique_labels, average=None)
class_recall = metrics.recall_score(test_labels, class_predictions, labels=unique_labels, average=None)

pd.DataFrame({'Class-specific precision':class_precision, 'Class-specific recall':class_recall})

Given the above class-specific precision and recall values the average values of these metrics can be computed as a weighted sum. The frequency of the test labels $n_i$ need not be the same or balanced. The weight, $w_i = n_i/\sum_j n_j$ is the decimal fraction of the 

\begin{align}   
average\ precision &= \sum_i w_i * precision_i\\ 
average\ recall &= \sum_i w_i * recall_i\\ 
w_i &= \frac{n_i}{\sum_j n_j}
\end{align} 

> **Exercise 4-5:** Now you will compute compute and display the average precision and recall scores for the model by creating and executing code in the cell below. 
> 1. Display the frequency of the unique labels, in the `label_counts` variable computed in the previous cell.   
> 2. Compute and display the weighted average of the precision and recall metrics.    

In [ ]:
## Put your code below  





> Answer the following questions in 1 to 3 well chosen sentences:  
> 1. Referring to the class-specific precision and recall computed above, what does the variation in the precision and recall scores of the different digit classes tell you about the performance of the classifier?   
> 2. Average precision and recall scores are often used to summarize the performance of multi-class classifiers. In this case, do you think the summary is reasonable, or does it loose too much information? 
> **End of exercise.**

> **Answers:**
> 1.     
> 2.    

> **Exercise 4-6:** To better understand the source of errors from the classifier model it is useful to examine some details of the erroneously classified cases. You will now do the following:    
> 1. Create an index vector of the erroneously classified cases.     
> 2. Create a Pandas data frame containing the class probabilities for the first 25 misclassified cases encountered. Hint, it will improve the readability of your display if you round to 3 decimal places.    
> 3. Append a column of the corresponding test labels to the data frame.  
> 4. Append a column showing the predicted class to the data frame. 
> 5. Display the data frame.    
> 6. Create a a $5 \times 5$ grid display the first 25 gray-scale erroneously classified images. Give each image display a title with the label for that image and the predicted class. *Hint,* use a large display area.  

In [ ]:
## Your code goes below                                                                                                                                                                                                                     







In [ ]:
fig, ax = plt.subplots(5, 5, figsize=(15,15))
ax = ax.flatten()

## Your code goes below






> Examine and compare the images of the misclassified digits to the probabilities of class assignment in the printed table and answer these questions:   
> 1. Notice that some of the digits shown are very poorly formed. Find a few such examples and then examine the category probabilities.    
> 2. Given the image and the probabilities describe why you think the error might have occurred?     
> **End of exercise.**     

> **Answers:**    

> **Exercise 4-7:** You may well wonder how one should choose the regularization hyperparameter for the model. The process of finding the best model for an application is know as **model selection.** In this case, answer is to perform a **hyperparameter search**. Now, do the following:   
> 1. Define a list of hyperparameter candidates, $[0.003, 0.01,0.03,0.1,0.3,1.0]$, the argument C.
> 2. Define 2 empty lists for accumulating train and test accuracy values.    
> 3. Iterate over the hyperparameter values. For each value fit a Logistic Regression model, compute the class predictions using both train and test data, and then append the train and test accuracy of the model to the associated list.   
> 4. Outside the loop compute lists (or arrays) of the train and test error rates for your model as $1.0 - accuracy$.
> 5. Display the accuracy and test error rate for each value of C.  
> 6. Execute your code, which may take some time.   

In [ ]:
np.random.seed(7890)
## Put your code below






> 6. On a reasonable size plot area, display a graph of the hyperparameter values on the horizontal axis and the train and test errors on the vertical axis. Include a legend on your plot so you can tell which line is the test and which is the train error. *Hint:* Your plot will be easier to interpret if you use a log scale on the horizontal axis using `ax.set_xscale('log')`.  

In [ ]:
## Put your code below






> Examine the graph and answer the following questions:
> 1. Given the results, why do you think the hyperparameter used was selected for the model?   
> 2. What does divergence between test and training error with increasing hyperparameter value tell you about improving the generalizeability of the model?   
> **End of exercise.** 

> **Answers:**
> 1.         
> 2.     

With the hyperparameter search completed we have completed the important process of **model selection**. We can now construct and test an optimal model. The code in the cell below constructs and performs evaluation on a mulit-class logistic regression model using this optimal L2 regularization hyperparameter value. Execute this code and examine the results.   

In [ ]:
C = 0.03
LinearClassifier = sklm.LogisticRegression(penalty='l2', C=C, multi_class='multinomial', solver ='newton-cg').fit(train_images, train_labels)

class_predictions = LinearClassifier.predict(test_images)
print('Overall accuracy = ' + str(metrics.accuracy_score(test_labels, class_predictions)))
print('Average precision = ' + str(weighted_average(class_precision)))
print('Average recall = ' + str(weighted_average(class_recall)))

confusion_matrix = metrics.confusion_matrix(test_labels, class_predictions)   
print(pd.DataFrame(confusion_matrix))
p = plt.imshow(np.log(np.divide(confusion_matrix + 1.0, np.sum(confusion_matrix, axis=1))))
cb = plt.colorbar(p)
_=cb.set_label('Log count')

In this case, the results are only slightly different from the model constructed with our initial regularization hyperparameter choice. This is not an unusal situation. One should not expect dramatic improvements solely from hyperparameter optimization.  However, optimizing model hyperparameters can lead to small, but useful, improvements in ML model performance.     

## Summary

We have investigated some key points of applying ML to CV problems in this lesson: 
1. The formulation of linear machine learning models. We will apply this formulation to CV problems beyond classification in subsequent lessons.     
2. A basic machine learning workflow with the following steps,   
   - Data preparation,    
   - Data repreresentation for the model,     
   - Model construction,    
   - model evaluation,    
   - and model selection.   
3. Formulation of CV features for machine learning. In this case, we used a basic approach of flattening the gray scale pixels of each image into a vector.    
4. The relationship between bias, variance and model capacity was explored through the use of regularization. Model selection was done by hyperparameter searching.  
5. Theory of binary classifiers was explored briefly.  
6. Theory of multi-class classifiers was applied to the digit classification problem. The results are probabilities for each class, with the largest probability selected as the result.    
7. Multi-class classifier evaluation was performed.     


#### Copyright  2018, 2019, 2020, 2021, 2022, 2023, 2024, Stephen F Elston. All rights reserved.  